In [1]:
import lancedb


/Users/enjalot/code/latent-scope-modal/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
db = lancedb.connect("/Users/enjalot/latent-scope-demo/ls-state-legislators-2024/lancedb")
table = db.open_table("scopes-001")
print(table.schema)





index: int64
text: string
NameLastFirstM: string
StateAbbrev: string
Chamberul: string
District: string
PartyAbbrev: string
Gender: string
RaceCoded: string
HighestDegree: string
JobTitle: string
JobOrgName: string
JobOrgDescr: string
WRKCL: int64
UNION: int64
SAFTY: int64
K12: int64
BUSNS: int64
FARM: int64
GOVT: int64
INSUR: int64
RLEST: int64
BANKS: int64
HGHED: int64
LAWJD: int64
DCTR: int64
FOSFU: int64
GREEN: int64
ELECT: int64
vacant: int64
YearBorn: double
idno: double
x: float
y: float
tile_index_32: int64
tile_index_64: int64
tile_index_128: int64
cluster: int64
raw_cluster: int64
label: string
deleted: bool
vector: fixed_size_list<item: float>[768]
  child 0, item: float
sae_indices: list<item: int64>
  child 0, item: int64
sae_acts: list<item: double>
  child 0, item: double


RuntimeError: lance error: LanceError(Schema): Column name does not exist, /Users/runner/.cargo/registry/src/index.crates.io-6f17d22bba15001f/lance-core-0.22.0/src/datatypes/schema.rs:191:31

In [81]:
column_name = f'`Gender`'
table.search().where(f"{column_name} = \'F\'").select(["text"]).to_list()

[{'text': 'Retail and Sales Management   '},
 {'text': 'CEO Galvin Educational Consulting Educational consultation'},
 {'text': 'Founder Wild Awake Helps gorw socially reponsible companies '},
 {'text': 'Legistative aid Alaska state legislature State government'},
 {'text': 'Science Teacher Keet Gooshi Heen Elementary School Elementary School'},
 {'text': 'Civil and Environmental Engineer Environmental Management Inc Environmantal consulting and engineering firm'},
 {'text': ' US Army '},
 {'text': 'President Ethos Construction Services, LLC a firm focused on unique property management'},
 {'text': 'Former staff member Mayor of Wasilla Local government '},
 {'text': 'Founder Great Alaska Schools Non - partisian Coialation advocating for a quality public education for every child '}]

In [82]:
table.search().select(["Gender"]).to_list()

[{'Gender': 'M'},
 {'Gender': 'M'},
 {'Gender': 'F'},
 {'Gender': 'M'},
 {'Gender': 'M'},
 {'Gender': 'F'},
 {'Gender': 'M'},
 {'Gender': 'F'},
 {'Gender': 'M'},
 {'Gender': 'M'}]

In [13]:
# Print the raw bytes/string representation of the values
import json
print(json.dumps([row['Gender'] for row in table.search().select(["Gender"]).limit(5).to_list()], indent=2))

[
  "M",
  "M",
  "F",
  "M",
  "M"
]


In [14]:
# Try with reversed quotes
table.search().where('\"Gender\" = "F"').select(["Gender", "text"]).to_list()

[]

In [15]:
# Try 1: Using double quotes for both
table.search().where('"Gender" = "F"').select(["Gender", "text"]).to_list()



[]

In [25]:
# Try 2: Using IS operator instead of =
table.search().where("\"Gender\" IN ('F')").select(["Gender", "text"]).to_list()


[]

In [17]:

# Try 3: Let's also try a different comparison to verify the where clause works at all
table.search().where('"Gender" IS NOT NULL').select(["Gender", "text"]).limit(5).to_list()

[{'Gender': 'M',
  'text': 'Economics and History teacher Ketchikan High School State High School'},
 {'Gender': 'M', 'text': 'Vice president KYMG Radio Radio station'},
 {'Gender': 'F', 'text': 'Retail and Sales Management   '},
 {'Gender': 'M',
  'text': 'Executive director Frontier Tutoring A tutoring service that helps high school students raise their test scores and grades'},
 {'Gender': 'M',
  'text': 'Former associate attorney - Partner Law Offices of G.R. Eschbacher Corporate law firm that offers counseling and substance abuse recovery programs'}]

In [20]:
table.schema

index: int64
text: string
NameLastFirstM: string
StateAbbrev: string
Chamberul: string
District: string
PartyAbbrev: string
Gender: string
RaceCoded: string
HighestDegree: string
JobTitle: string
JobOrgName: string
JobOrgDescr: string
WRKCL: int64
UNION: int64
SAFTY: int64
K12: int64
BUSNS: int64
FARM: int64
GOVT: int64
INSUR: int64
RLEST: int64
BANKS: int64
HGHED: int64
LAWJD: int64
DCTR: int64
FOSFU: int64
GREEN: int64
ELECT: int64
vacant: int64
YearBorn: double
idno: double
x: float
y: float
tile_index_32: int64
tile_index_64: int64
tile_index_128: int64
cluster: int64
raw_cluster: int64
label: string
deleted: bool
vector: fixed_size_list<item: float>[768]
  child 0, item: float
sae_indices: list<item: int64>
  child 0, item: int64
sae_acts: list<item: double>
  child 0, item: double

In [22]:
table.list_indices()


[Index(IvfPq, columns=["vector"], name="vector_idx"),
 Index(BTree, columns=["cluster"], name="cluster_idx"),
 Index(LabelList, columns=["sae_indices"], name="sae_indices_idx")]

In [68]:
# DataFusion style: double quotes for identifier (column name), single quotes for string literal
table.search().where('"PartyAbbrev" = \'D\'').select(["PartyAbbrev", "text"]).to_list()

[]

In [67]:
# Try a basic IS NOT NULL to verify where clause works
table.search().where('"PartyAbbrev" IS NOT NULL').select(["PartyAbbrev", "text"]).limit(5).to_list()

[{'PartyAbbrev': 'I',
  'text': 'Economics and History teacher Ketchikan High School State High School'},
 {'PartyAbbrev': 'R', 'text': 'Vice president KYMG Radio Radio station'},
 {'PartyAbbrev': 'R', 'text': 'Retail and Sales Management   '},
 {'PartyAbbrev': 'Nonpartisan',
  'text': 'Executive director Frontier Tutoring A tutoring service that helps high school students raise their test scores and grades'},
 {'PartyAbbrev': 'D',
  'text': 'Former associate attorney - Partner Law Offices of G.R. Eschbacher Corporate law firm that offers counseling and substance abuse recovery programs'}]

In [69]:
# Try a basic IS NOT NULL to verify where clause works
column_name = f'"PartyAbbrev"'
asdf = table.search().where(f"\"PartyAbbrev\" = 'D'").select(["PartyAbbrev", "text"])



In [70]:
dir(asdf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_columns',
 '_ef',
 '_limit',
 '_offset',
 '_prefilter',
 '_query_to_vector',
 '_resolve_query',
 '_table',
 '_text',
 '_use_index',
 '_vector',
 '_where',
 '_with_row_id',
 'create',
 'explain_plan',
 'limit',
 'offset',
 'rerank',
 'select',
 'text',
 'to_arrow',
 'to_batches',
 'to_df',
 'to_list',
 'to_pandas',
 'to_polars',
 'to_pydantic',
 'vector',
 'where',
 'with_row_id']

In [71]:
asdf._where

'"PartyAbbrev" = \'D\''

In [72]:
asdf._columns

['PartyAbbrev', 'text']

In [73]:
asdf.to_list()

[]

In [79]:
column_name = f'`PartyAbbrev`'
q = table.search().where(f'{column_name} LIKE \'D%\'').select(["text"]).to_list()
q


[{'text': 'Former associate attorney - Partner Law Offices of G.R. Eschbacher Corporate law firm that offers counseling and substance abuse recovery programs'},
 {'text': 'Founder Wild Awake Helps gorw socially reponsible companies '},
 {'text': 'Former workforce development coordinator Alaska Department of Labor and Workforce Development '},
 {'text': 'Lawyer Law office of Clifford J Groh Private practice'},
 {'text': 'Legistative aid Alaska state legislature State government'},
 {'text': 'Physician Asistant Veterans Affairs Hospital Hospital'},
 {'text': 'Civil and Environmental Engineer Environmental Management Inc Environmantal consulting and engineering firm'},
 {'text': 'Founder Great Alaska Schools Non - partisian Coialation advocating for a quality public education for every child '},
 {'text': 'Teacher  Denali Elementary Elemntary school'},
 {'text': 'Chief of staff Adam Wool Alaska State Legislature'}]

In [80]:
table.search().where(f"{column_name} = 'D'")._where

"`PartyAbbrev` = 'D'"